In [1]:
import os
import sys

sys.path.append(os.path.abspath(".."))

In [2]:
%pip install belashovplot

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [43]:
from belashovplot import TiledPlot
from utilities import *
from elements.modulators import Lens, PhaseModulator
from elements.propagators import FurrierPropagation
from elements.composition import CompositeModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from tqdm import tqdm
from math import sin, sqrt
import torch

### Настройка параметров системы

In [35]:
length = 20.0E-3
wavelength = 500E-9
b = 20.0E-6
cppp = 3
modulators_amount = 5

pixels = int(length/b)
N = int(pixels*cppp)
distance = length * sqrt((b/wavelength)**2-1)

print(f"Количество вычислительных пикселей: {N}")
print(f"Количество пикселей маски:          {pixels}")
print(f"Размер пикселя:                     {engineering(b, 'м')}")
print(f"Расстояние между слоями:            {engineering(distance, 'м')}")
print(f"Количество модуляторов:             {modulators_amount}")

Количество вычислительных пикселей: 2997
Количество пикселей маски:          999
Размер пикселя:                     20.0 мкм
Расстояние между слоями:            799.75 мм
Количество модуляторов:             5


### Настройка некогерентности

In [39]:
spatial_coherence = 50.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 100

print(f"Временная когерентность:        {engineering(time_coherence, 'с')}")
print(f"Время релаксации:               {engineering(time_relaxation, 'c')}")
print(f"Пространственная когерентность: {engineering(spatial_coherence, 'м')}")
print(f"Пространственный масштаб:       {engineering(length, 'м')}")

Временная когерентность:        10.0 нс
Время релаксации:               1.0 мкc
Пространственная когерентность: 50.0 мкм
Пространственный масштаб:       20.0 мм


### Инициализация модулей и оптической модели

In [41]:
propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.7)
modulators = [PhaseModulator(N, length, pixels) for i in range(modulators_amount)]
elements = [modulators[0]]
for modulator in modulators[1:]:
    elements.append(propagation)
    elements.append(modulator)

incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, pixels, length)
chunker = CudaMemoryChunker()

model = CompositeModel(*elements)
model.wrap(chunker)
model.wrap(incoherent)

### Выбор набора данных

In [51]:
dataset = Dataset('Flowers', 100, N, N, torch.complex64)
dataset.train
dataset.test

100%|██████████| 344862509/344862509 [00:06<00:00, 54911204.17it/s]


Extracting /home/jupyter/work/resources/OpticalEncoder/datasets/flowers-102/102flowers.tgz to /home/jupyter/work/resources/OpticalEncoder/datasets/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1024095.63it/s]


100%|██████████| 14989/14989 [00:00<00:00, 27086782.70it/s]


### Тестирование модели